In [37]:
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=20, use_memory_fs=False)

import os
from ete3 import Tree
import random

import sys
sys.path.append("/groups/itay_mayrose/halabikeren/tmp/ploidb/data_processing/")
from check_tree_monophyly import get_largest_monophyletic_group

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [38]:
ccdb_unresolved_names_path = "/groups/itay_mayrose/halabikeren/PloiDB/name_resolution/ccdb_unresolved_names.csv"
allotb_unresolved_names_path = "/groups/itay_mayrose/halabikeren/PloiDB/name_resolution/ALLOTB_tree_unresolved_names.csv"
allotb_tree_path = "/groups/itay_mayrose/halabikeren/PloiDB/trees/ALLMB.tre" 

ccdb_resolved_names_path = "/groups/itay_mayrose/anatshafir1/ploidDB/rotl/ALLOTB/stats/resolved_name_mapping_ccdb.csv"
tree_resolved_names_path = "/groups/itay_mayrose/anatshafir1/ploidDB/rotl/ALLOTB/stats/resolved_name_mapping_tree.csv"
intersection_resolved_names_path = "/groups/itay_mayrose/anatshafir1/ploidDB/rotl/ALLOTB/stats/resolved_name_mapping_intersection.csv"

unresolved_tree_path = "/groups/itay_mayrose/halabikeren/PloiDB/trees/ALLOTB.tre"
resolved_tree_path = "/groups/itay_mayrose/halabikeren/PloiDB/trees/resolved_ALLOTB.nwk"
selected_tree_leaves_path = "/groups/itay_mayrose/halabikeren/PloiDB/trees/selected_ALLOTB_original_names.csv"

unresolved_ccdb_path = "/groups/itay_mayrose/halabikeren/PloiDB/ccdb/all_data.csv"
resolved_ccdb_path = "/groups/itay_mayrose/halabikeren/PloiDB/ccdb/resolved_data.csv"

In [39]:
ccdb_unresolved_names = pd.read_csv(ccdb_unresolved_names_path)
tree_unresolved_names = pd.read_csv(allotb_unresolved_names_path)

In [40]:
def process_tree(path: str) -> Tree:
    tree = Tree(path, format=1)
    names = set()
    for leaf in tree.get_leaves():
        leaf_name = leaf.name.lower().replace("_", " ")
        if leaf_name in names:
            print(f"{leaf_name} already in tree")
            leaf.detach()
        else:
            leaf.name = leaf_name
            names.add(leaf_name)
    return tree
    
unresolved_tree = process_tree(unresolved_tree_path)

In [43]:
def correct_name(name: str) -> str:
    name = name.split("(")[0]
    name = name.split(",")[0]
    if name.endswith(" "):
        name = name[:-1]
    return name

def process_resolved_names(path: str) -> pd.DataFrame:
    resolved_names = pd.read_csv(path).drop("Unnamed: 0", axis=1)
    resolved_names.rename(columns={"search_string": "corrected_original_name", 
                                   "unique_name": "resolved_name"}, inplace=True)
    resolved_names.original_name = resolved_names.original_name.str.lower()
    resolved_names.corrected_original_name = resolved_names.corrected_original_name.str.lower()
    resolved_names.resolved_name = resolved_names.resolved_name.str.lower()
    resolved_names["corrected_resolved_name"] = resolved_names.resolved_name.apply(correct_name)
    resolved_names.sort_values("ott_id", inplace=True)
    resolved_names = resolved_names[["original_name", "corrected_original_name", "resolved_name", "corrected_resolved_name", "ott_id"]]
    return resolved_names

ccdb_resolved_names = process_resolved_names(path=ccdb_resolved_names_path)
tree_resolved_names = process_resolved_names(path=tree_resolved_names_path)
intersection_resolved_names = process_resolved_names(path=intersection_resolved_names_path)
print(f"ccdb coverage = {np.round(ccdb_resolved_names.shape[0]/ccdb_unresolved_names.shape[0]*100,2)}%")
print(f"tree coverage = {np.round(tree_resolved_names.shape[0]/tree_unresolved_names.shape[0]*100,2)}%")
print(f"intersection coverage = {np.round(len(intersection_resolved_names.ott_id.unique())/len(tree_resolved_names.ott_id.unique())*100,2)}%")

ccdb coverage = 90.0%
tree coverage = 96.96%
intersection coverage = 19.34%


In [44]:
def add_resolved_names(tree: Tree, resolved_names: pd.DataFrame):
    orig_to_resolved = resolved_names.set_index("original_name")["corrected_resolved_name"].to_dict()
    for leaf in tree.get_leaves():
        leaf.add_feature(pr_name="corrected_resolved_name", pr_value=orig_to_resolved[leaf.name])


def select_orig_name(record: pd.Series, tree: Tree, ccdb_orig_names: list[str]) -> str:
    resolved_name = record.corrected_resolved_name
    orig_names = record.original_names
    if len(orig_names) == 1:
        return orig_names[0]
    is_monophyletic, clade_type, monophyly_violators = tree.check_monophyly(values=[resolved_name], target_attr="resolved_name")
    if is_monophyletic:
        return random.choice(orig_names)
    else:
        lca = tree
        try:
            lca = tree.get_common_ancestor([l for l in tree.get_leaves() if l.corrected_resolved_name == resolved_name])
        except Exception as e:
            print(f"Couldn't find lca for {', '.join(orig_names)} due to error {e}, and will thus set the root as lca")
        mono_root, mono_size, is_larger_than_rest = get_largest_monophyletic_group(root=lca, property_value=resolved_name, property_name="resolved_name")
        if is_larger_than_rest:
            return random.choice(mono_root.get_leaf_names())
        else:
            orig_names_in_ccdb = [name for name in orig_names if name in ccdb_orig_names]
            if len(orig_names_in_ccdb) == 1:
                return orig_names_in_ccdb[0]
    return random.choice(orig_names)
  
    
def resolve_tree(tree: Tree, resolved_names: pd.DataFrame, ccdb_orig_names: list[str], resolved_to_selected_orig_path: str) -> Tree:
    resolved_tree = tree.copy()
    print(f"original number of leaves = {len(resolved_tree.get_leaf_names()):,}")
    leaves_to_keep = resolved_names.original_name.tolist()
    resolved_tree.prune(leaves_to_keep)
    add_resolved_names(tree=resolved_tree, resolved_names=resolved_names)
    print(f"number of leaves after prunning of unmapped names to ccdb = {len(resolved_tree.get_leaf_names()):,}")
    if os.path.exists(resolved_to_selected_orig_path):
        resolved_to_orig = pd.read_csv(resolved_to_selected_orig_path)
    else:
        resolved_to_orig = resolved_names.groupby("resolved_name")["original_name"].apply(lambda names: names.tolist()).reset_index().rename(columns={"original_name": "original_names"})
        resolved_to_orig["selected_original_name"] = resolved_to_orig.parallel_apply(lambda record: select_orig_name(record=record, tree=resolved_tree, ccdb_orig_names=ccdb_orig_names), axis=1)
        resolved_to_orig.to_csv(resolved_to_selected_orig_path, index=False)
    resolved_tree.prune(resolved_to_orig["selected_original_name"].tolist())
    print(f"number of leaves after removal of leaves with identical resolved name = {len(resolved_tree.get_leaf_names()):,}")
    for leaf in resolved_tree.get_leaves():
        leaf.name = leaf.corrected_resolved_name
    return resolved_tree


resolved_tree = resolve_tree(tree=unresolved_tree, 
                             resolved_names=intersection_resolved_names, 
                             ccdb_orig_names=ccdb_resolved_names.original_name.tolist(), 
                             resolved_to_selected_orig_path=selected_tree_leaves_path)
resolved_tree.write(outfile=resolved_tree_path)

original number of leaves = 353,185
number of leaves after prunning of unmapped names to ccdb = 74,569
number of leaves after removal of leaves with identical resolved name = 59,909
